In [ ]:
# Install dependencies
!pip install ultralytics pyyaml

# Imports
import numpy as np
import pandas as pd
import os
import random
import shutil
import glob
import yaml
import torch
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import cv2
from PIL import Image
from pathlib import Path
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# Mount Google Drive (upload your dataset to Drive first)
from google.colab import drive
drive.mount('/content/drive')

# ================== PATHS CONFIGURATION ==================
# Update these paths according to your dataset location
DATASET_PATH = "/content/drive/MyDrive/face-mask-detection"
IMAGE_DIR = os.path.join(DATASET_PATH, "images")
ANNOTATION_DIR = os.path.join(DATASET_PATH, "annotations")

# Output directories
WORK_DIR = "/content/datasets"
os.makedirs(WORK_DIR, exist_ok=True)

# ================== DATA PROCESSING ==================
# Load and parse annotations
annotations = list(Path(ANNOTATION_DIR).glob(r'**/*.xml'))

class_id = {
    "with_mask": 0,
    "mask_weared_incorrect": 1,
    "without_mask": 2
}

data_dict = {
    'filename': [], 'label': [], 'class_id': [],
    'width': [], 'height': [], 'bboxes': []
}

for annotation_path in annotations:
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    filename = root.find('filename').text

    for obj in root.findall("object"):
        label = obj.find("name").text
        bndbox = obj.find('bndbox')

        bbox = [
            int(bndbox.find('xmin').text),
            int(bndbox.find('ymin').text),
            int(bndbox.find('xmax').text),
            int(bndbox.find('ymax').text)
        ]

        size = root.find('size')
        data_dict['filename'].append(filename)
        data_dict['width'].append(int(size.find('width').text))
        data_dict['height'].append(int(size.find('height').text))
        data_dict['label'].append(label)
        data_dict['class_id'].append(class_id[label])
        data_dict['bboxes'].append(bbox)

df_data = pd.DataFrame(data_dict)

# ================== DATA SPLITTING ==================
# Create train/val/test directories
train_path = os.path.join(WORK_DIR, "train")
valid_path = os.path.join(WORK_DIR, "valid")
test_path = os.path.join(WORK_DIR, "test")

os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Split dataset
train_files, test_files = train_test_split(df_data.filename.unique(), test_size=0.2, random_state=23)
train_files, valid_files = train_test_split(train_files, test_size=0.15, random_state=23)

# ================== DATA PREPARATION ==================
def pascal_voc_to_yolo(bbox, img_w, img_h):
    x_min, y_min, x_max, y_max = bbox
    x_center = ((x_min + x_max) / 2) / img_w
    y_center = ((y_min + y_max) / 2) / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h
    return [x_center, y_center, width, height]

def prepare_dataset(files, target_dir):
    # Copy images
    for file in files:
        src = os.path.join(IMAGE_DIR, file)
        dst = os.path.join(target_dir, file)
        shutil.copy(src, dst)

    # Create labels
    for file in files:
        file_stem = Path(file).stem
        label_path = os.path.join(target_dir, f"{file_stem}.txt")

        file_data = df_data[df_data['filename'] == file]
        with open(label_path, 'w') as f:
            for _, row in file_data.iterrows():
                yolo_bbox = pascal_voc_to_yolo(row['bboxes'], row['width'], row['height'])
                line = f"{row['class_id']} {' '.join(map(str, yolo_bbox))}"
                f.write(line + '\n')

# Prepare all datasets
prepare_dataset(train_files, train_path)
prepare_dataset(valid_files, valid_path)
prepare_dataset(test_files, test_path)

# ================== YAML CONFIG ==================
yaml_content = f"""
train: {train_path}
val: {valid_path}
test: {test_path}
nc: 3
names: {list(class_id.keys())}
"""

with open('facemask.yaml', 'w') as f:
    f.write(yaml_content)

# ================== TRAINING ==================
model = YOLO("yolov8n.pt")
model.train(data="facemask.yaml", epochs=20, project="/content/runs")

# ================== VALIDATION ==================
model.val(data="facemask.yaml")

# ================== PREDICTION ==================
test_images = glob.glob(os.path.join(test_path, "*.png"))
results = model.predict(test_images, save=True, project="/content/runs/predictions")

# Display predictions
for img_path in glob.glob("/content/runs/predictions/*.jpg"):
    img = Image.open(img_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=facemask.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optim

train: Scanning /content/datasets/train... 579 images, 0 backgrounds, 0 corrupt: 100%|██████████| 579/579 [00:00<00:00, 762.91it/s]

train: New cache created: /content/datasets/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3039.0±925.5 MB/s, size: 401.2 KB)


val: Scanning /content/datasets/valid... 103 images, 0 backgrounds, 0 corrupt: 100%|██████████| 103/103 [00:00<00:00, 802.11it/s]

...


20 epochs completed in 2.979 hours.
Optimizer stripped from /content/runs/train/weights/last.pt, 6.2MB
Optimizer stripped from /content/runs/train/weights/best.pt, 6.2MB

Validating /content/runs/train/weights/best.pt...
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:25<00:00,  6.31s/it]


                   all        103        615      0.803      0.762      0.799      0.503
             with_mask         97        455      0.832      0.914      0.936      0.644
 mask_weared_incorrect         14         23      0.805       0.54      0.603      0.353
          without_mask         36        137      0.772      0.832      0.859      0.512
Speed: 2.8ms preprocess, 208.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/runs/train
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1278.3±429.6 MB/s, size: 541.1 KB)


val: Scanning /content/datasets/valid.cache... 103 images, 0 backgrounds, 0 corrupt: 100%|██████████| 103/103 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:26<00:00,  3.73s/it]


                   all        103        615      0.792      0.738      0.796      0.502
             with_mask         97        455      0.832      0.912      0.934      0.643
 mask_weared_incorrect         14         23      0.784      0.478      0.597      0.351
          without_mask         36        137      0.759      0.825      0.855      0.511
Speed: 1.8ms preprocess, 210.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/runs/train2

0: 640x640 1 with_mask, 358.4ms
1: 640x640 1 with_mask, 358.4ms
2: 640x640 2 with_masks, 358.4ms
3: 640x640 1 with_mask, 358.4ms
4: 640x640 20 with_masks, 1 mask_weared_incorrect, 2 without_masks, 358.4ms
5: 640x640 1 without_mask, 358.4ms
6: 640x640 2 with_masks, 358.4ms
7: 640x640 4 with_masks, 358.4ms
8: 640x640 8 with_masks, 4 without_masks, 358.4ms
9: 640x640 1 with_mask, 358.4ms
10: 640x640 1 with_mask, 358.4ms
11: 640x640 3 with_masks, 1 without_mask, 358.4ms
12: 640x640 3 with_masks, 1 without_mask, 358.4ms
